In [2]:
!docker build . --quiet --tag code-executor:latest

sha256:34a84cac20172268a4e67eedf0b101f63b0d21b546bb4b458f30f76847ae1c76


In [3]:
!docker run -d -p 8080:8080 --name test-code-executor code-executor:latest --host 0.0.0.0

3aa1ae260864f134b12ad72bcbfbb9e70105a8801dace45097a755e625081898


In [3]:
!docker logs test-code-executor

Error response from daemon: No such container: test-code-executor


In [7]:
!docker kill test-code-executor && docker rm test-code-executor

test-code-executor
test-code-executor


In [6]:
!docker rm test-code-executor

Error response from daemon: You cannot remove a running container 5f99f754e027c20b7a53f707579040df2419fbe51ab571172bff0ceaf79223f3. Stop the container before attempting removal or force remove


In [6]:
!docker context ls

NAME                             DESCRIPTION                               DOCKER ENDPOINT                                              ERROR
default                          Current DOCKER_HOST based configuration   unix:///var/run/docker.sock                                  
m5.large_i-02880917500cf72a3 *                                             tcp://ip-172-31-79-18.ap-southeast-2.compute.internal:1111   


In [4]:
host_ip = "ip-172-31-79-18.ap-southeast-2.compute.internal"

In [5]:
%%bash -s $host_ip
curl ${1}:8080/ping

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    22  100    22    0     0   3112      0 --:--:-- --:--:-- --:--:--  3666


{"Health_Check":"200"}

In [6]:
%%bash -s $host_ip
curl ${1}:8080/execute_code -X POST -d '{"code": "print(\"Hello ok)"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   172  100   143  100    29  10030   2034 --:--:-- --:--:-- --:--:-- 12285


{"output":"Cell In[1], line 1\n    print(\"Hello ok)\n          ^\nSyntaxError: unterminated string literal (detected at line 1)","error":true}

In [7]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "print(\"Hello ok\")", "timeout": 10}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    81  100    35  100    46   2004   2634 --:--:-- --:--:-- --:--:--  4764


{"output":"Hello ok","error":false}

In [8]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "a=10", "timeout": 10}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    58  100    27  100    31   3287   3774 --:--:-- --:--:-- --:--:--  7250


{"output":"","error":false}

In [9]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "print(a)", "timeout": 10}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    64  100    29  100    35   3200   3862 --:--:-- --:--:-- --:--:--  8000


{"output":"10","error":false}

In [10]:
%%bash -s $host_ip
curl -X POST ${1}:8080/restart_kernel -d '{"code": "print(\"Hello ok\")", "timeout": 10, "kernel_name": "ir"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   114  100    47  100    67    375    534 --:--:-- --:--:-- --:--:--   912


{"output":"Kernel ir restarted!","error":false}

In [11]:
%%bash -s $host_ip
curl ${1}:8080/list_kernel_specs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   349  100   349    0     0  37934      0 --:--:-- --:--:-- --:--:-- 43625


{"python3":{"display_name":"Python 3 (ipykernel)","language":"python"},"ir":{"display_name":"R","language":"R"},"julia-1.9":{"display_name":"Julia 1.9.3","language":"julia"},"javascript":{"display_name":"JavaScript (Node.js)","language":"javascript"},"bash":{"display_name":"Bash","language":"bash"},"java":{"display_name":"Java","language":"java"}}

In [41]:
java_code = '\npublic class Main {\n  public static void main(String[] args) {\n    System.out.println(\"Hello World!\");  \n  }\n}\n'

In [42]:
import requests
import json
data = {"code": java_code, "timeout": 10, "kernel_name": "java"}
print(data)
requests.post(f"http://{host_ip}:8080/execute_code", data=json.dumps(data)).text

{'code': '\npublic class Main {\n  public static void main(String[] args) {\n    System.out.println("Hello World!");  \n  }\n}\n', 'timeout': 10, 'kernel_name': 'java'}


'{"output":"","error":false}'

In [43]:
import requests
import json
data = {"code": "Main.main(new String[] {\"\"})", "timeout": 10, "kernel_name": "java"}
print(data)
requests.post(f"http://{host_ip}:8080/execute_code", data=json.dumps(data)).text

{'code': 'Main.main(new String[] {""})', 'timeout': 10, 'kernel_name': 'java'}


'{"output":"Hello World!","error":false}'

In [21]:
%%bash -s $host_ip $java_code
echo ${2}
data='{"code": "'${2}'", "timeout": 10, "kernel_name": "java"}'
echo $data
curl -X POST ${1}:8080/execute_code -d ${data}

public
{"code": "public", "timeout": 10, "kernel_name": "java"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    29  100    21  100     8   5312   2023 --:--:-- --:--:-- --:--:--  9666
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: "public",
curl: (3) URL using bad/illegal format or missing URL
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: 10,
curl: (3) URL using bad/illegal format or missing URL
curl: (3) unmatched close brace/bracket in URL position 7:
"java"}
      ^


Internal Server Error

CalledProcessError: Command 'b'echo ${2}\ndata=\'{"code": "\'${2}\'", "timeout": 10, "kernel_name": "java"}\'\necho $data\ncurl -X POST ${1}:8080/execute_code -d ${data}\n'' returned non-zero exit status 3.

In [219]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "echo 1", "timeout": 10, "kernel_name": "bash"}'

{"output":"1","error":false}

In [12]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "(int) eval(\"1+2\")+3", "timeout": 10, "kernel_name": "java"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    99  100    28  100    71     26     66  0:00:01  0:00:01 --:--:--    93


{"output":"6","error":false}

In [246]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "console.log(\"Hello World\");", "timeout": 10, "kernel_name": "javascript"}'


{"output":"Hello World","error":false}

In [19]:
%%bash -s $host_ip
curl ${1}:8080/list_kernel_specs

curl: (6) Could not resolve host: ip-172-31-65-185.ap-southeast-2.compute.internal


In [45]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "numbers <- c(1, 4, 7, 3, 2)\navg <- mean(numbers)\nprint(avg)", "timeout": 10, "kernel_name": "ir"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   143  100    34  100   109    393   1260 --:--:-- --:--:-- --:--:--  1662


{"output":"[1] 3.4","error":false}

In [40]:
!A=$"numbers <- c(1, 4, 7, 3, 2)\\navg <- mean(numbers)\nprint(avg)"; echo $A

numbers <- c(1, 4, 7, 3, 2)\navg <- mean(numbers)\nprint(avg)


In [ ]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "print(\"Hello World!\")", "timeout": 10, "kernel_name": "ir"}'